The goal of this notebook is to find out the performance of the 5mo model when trying to predict the remaining 9mo. If it isn't very good, then the idea is to use this model to initialize a LwF model and use that to improve performance.

In [4]:
from importlib import reload
import numpy as np
import pandas as pd

import util
from types_ import DatasetConfiguration
import metrics
import estimators.linear as linear_estimators
import estimators.mtr as mtr_estimators
import plotting

from sklearn.model_selection import KFold
import metrics

In [ ]:
reload(util)
X, y, previous_recall_score, df = util.load_data(DatasetConfiguration(users=['user_14']))

Added 334517 logs
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneTy

unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported operand type(s) for *: 'float' and 'NoneType'
unsupported op

In [3]:

reload(metrics)
reload(mtr_estimators)
reload(linear_estimators)

SEEDS = [100,101]

estimator = None

def cross_validate_lomb_dataset(folds, seeds, estimator_constructor):
    global estimator
    
    y_trues                = []
    y_preds                = []
    previous_recall_scores = []
    deltas                 = []

    for seed in seeds:
        kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
        for train_index, test_index in kf.split(X):

            X_train = X.iloc[train_index]
            y_train = y.iloc[train_index]
            X_test  = X.iloc[test_index]
            y_test  = y.iloc[test_index]
            delta_train = X['delta'].iloc[train_index]
            delta_test  = X['delta'].iloc[test_index]
            previous_recall_score_test = previous_recall_score.iloc[test_index]

            estimator = estimator_constructor()
            estimator.fit(X_train, y_train, delta_train)

            y_pred = estimator.predict(X_test, delta_test)
            y_pred = np.clip(y_pred, 0.000001, 0.999999)

            y_trues.append(y_test)
            y_preds.append(y_pred)
            previous_recall_scores.append(previous_recall_score_test)
            deltas.append(delta_test)
    return metrics.LombEvaluation.from_CV(estimator.get_name(),
                                                y_trues, 
                                                y_preds, 
                                                previous_recall_scores, 
                                                deltas)

for estimator in linear_estimators.all_estimators+mtr_estimators.all_estimators:
    print(cross_validate_lomb_dataset(5, SEEDS, estimator).to_latex_tabular_row())

Linear Regression & 0.0980 & 0.0131 $\pm$ 0.0014 & 0.000286 $\pm$ 0.00017 \\
Logistic Regression & 0.106 & 0.0225 $\pm$ 0.0023 & 0.000478 $\pm$ 0.00024 \\
HLR & 0.140 & 0.0126 $\pm$ 0.0021 & 0.000334 $\pm$ 0.00022 \\
HLR & 0.134 & 0.0107 $\pm$ 0.0014 & 0.000130 $\pm$ 0.00021 \\


/home/charlie/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


MTR-Exp & 0.203 & 0.00880 $\pm$ 0.0011 & 0.000105 $\pm$ 0.00012 \\
MTR-ESq & 0.164 & 0.00769 $\pm$ 0.00072 & 0.000101 $\pm$ 0.00014 \\
MTR-Hyp & 0.203 & 0.00894 $\pm$ 0.00092 & 0.0000975 $\pm$ 0.00012 \\
MTR-HSq & 0.165 & 0.00764 $\pm$ 0.00074 & 0.0000952 $\pm$ 0.00015 \\
MTR-SWP & 0.151 & 0.00747 $\pm$ 0.00082 & 0.0000988 $\pm$ 0.00016 \\
